In [2]:
using CSV
using DataFrames
using PrettyTables
using StatsBase
include("./advgames.jl")
include("./advgames_analysis.jl")   

variance_inclinations (generic function with 1 method)

In [3]:
function run_simulation(df, num_simulations, numrounds, params_for_model, params_filename)
    for nsim in 1:num_simulations
        adata = [pr_coop]
        # model data to record
        mdata = [
            cc, 
            cd, 
            dc, 
            dd, 
            game_type,
            avg_incl,
            pr_adv_coop,
            pr_hyp_adv_c_play_d, 
            pr_hyp_adv_d_play_c, 
            perc_cooperative,
            perc_uncooperative,
            perc_middle,
            variance_inclinations
            ]
        # params is a dictionary with the parameters to be used in the simulation
        model, parameters = initialize(; params_for_model...)            
        #pretty_table(parameters, show_subheader=false)
        # write the parameters to a txt file
        open("$(params_filename)_$(nsim).txt", "w") do io
            pretty_table(io, parameters, show_subheader=false)
        end

        _, mdf = run!(model, dummystep, model_step!, numrounds; adata, mdata)
        
        # average the mdf avg_incl
        cai = mean(mdf.avg_incl)

        # proportion of the times perc_cooperative >= 0.75
        time_on_top = sum(mdf.perc_cooperative .>= 0.75) / length(mdf.perc_cooperative) 
        time_on_bottom = sum(mdf.perc_uncooperative .>= 0.75) / length(mdf.perc_cooperative) 

        if time_on_bottom == 0
            coop_degree = Inf
        else
            coop_degree = time_on_top / time_on_bottom
        end
        
        order_degree = time_on_top + time_on_bottom

        R = model.base_payouts[:C, :C]
        S = model.base_payouts[:C, :D]
        T = model.base_payouts[:D, :C]
        P = model.base_payouts[:D, :D]
        tolerance = parameters[:tolerance]
        num_defectors = parameters[:num_defectors]
        num_cooperators = parameters[:num_cooperators]
        num_neutral = parameters[:num_neutral]
        max_positive_pressure = parameters[:max_pos_neg_pressures][1]
        max_negative_pressure = parameters[:max_pos_neg_pressures][2]
        adv_mutation_rate = parameters[:a_m_mutation_rates][1]
        move_mutation_rate = parameters[:a_m_mutation_rates][2]
        weight_of_present_for_adv = parameters[:a_m_weights_of_present][1]
        weight_of_present_for_move = parameters[:a_m_weights_of_present][2]
        adv_reassess_frequency = parameters[:a_m_reassess_frequencies][1]
        move_reassess_frequency = parameters[:a_m_reassess_frequencies][2]
        move_increment = parameters[:m_increment]
        push!(df, [cai, coop_degree, order_degree, nsim, numrounds, R, S, T, P, tolerance, num_defectors, num_cooperators, num_neutral, max_positive_pressure, max_negative_pressure, adv_mutation_rate, move_mutation_rate, weight_of_present_for_adv, weight_of_present_for_move, adv_reassess_frequency, move_reassess_frequency, move_increment])
    end
end


run_simulation (generic function with 1 method)

In [4]:
numrounds = 500_000
num_simulations = 50
df = DataFrame(
    cai=Float64[],
    coop_degree=Float64[], 
    order_degree=Float64[], 
    numsim = Int[],
    numrounds=Int[],
    R=Float64[], 
    S=Float64[], 
    T=Float64[], 
    P=Float64[], 
    tolerance=Float64[], 
    num_defectors=Int[], 
    num_cooperators=Int[], 
    num_neutral=Int[], 
    max_positive_pressure=Float64[], 
    max_negative_pressure=Float64[], 
    adv_mutation_rate=Float64[], 
    move_mutation_rate=Float64[], 
    weight_of_present_for_adv=Float64[], 
    weight_of_present_for_move=Float64[], 
    adv_reassess_frequency=Float64[], 
    move_reassess_frequency=Float64[], 
    move_increment=Float64[]
)

params_for_model = Dict(
    :num_cooperators => 6, 
    :num_defectors => 6, 
    :num_neutral => 0,
    :max_pos_pressure => 6,
    :max_neg_pressure => 4,
    :adv_mutation_rate => 0.2,
    :move_mutation_rate => 0.15,
    :tolerance => 0.05,
    :payouts => Dict(
        (:C, :C) => 3.0,
        (:C, :D) => 0.0,
        (:D, :C) => 4.0,
        (:D, :D) => 1.0
        )
        )
run_simulation(df, num_simulations, numrounds, params_for_model, "./diff_initial_conditions/params")
CSV.write("diff_initial_conditions/diff_initial_conditions.csv", df)


"diff_initial_conditions/diff_initial_conditions.csv"

In [6]:
function generate_sequences(sum_val, length)
    result = []
    for i in 0:sum_val
        for j in 0:sum_val
            k = sum_val - i - j
            if k >= 0
                push!(result, [i, j, k])
            end
        end
    end
    return result
end

# Generate sequences of length 3 that sum to 12
initial_distributions_of_agents = generate_sequences(12, 3)

numrounds = 500_000
num_simulations = 1
df = DataFrame(
    cai=Float64[],
    coop_degree=Float64[], 
    order_degree=Float64[], 
    numsim = Int[],
    numrounds=Int[],
    R=Float64[], 
    S=Float64[], 
    T=Float64[], 
    P=Float64[], 
    tolerance=Float64[], 
    num_defectors=Int[], 
    num_cooperators=Int[], 
    num_neutral=Int[], 
    max_positive_pressure=Float64[], 
    max_negative_pressure=Float64[], 
    adv_mutation_rate=Float64[], 
    move_mutation_rate=Float64[], 
    weight_of_present_for_adv=Float64[], 
    weight_of_present_for_move=Float64[], 
    adv_reassess_frequency=Float64[], 
    move_reassess_frequency=Float64[], 
    move_increment=Float64[]
)

for (num_cooperators, num_defectors, num_neutral) in initial_distributions_of_agents
    println("$(num_cooperators) $(num_defectors) $(num_neutral)")
    params_for_model = Dict(
        :num_cooperators => num_cooperators, 
        :num_defectors => num_defectors, 
        :num_neutral => num_neutral,
        :max_pos_pressure => 6,
        :max_neg_pressure => 4,
        :adv_mutation_rate => 0.2,
        :move_mutation_rate => 0.15,
        :tolerance => 0.05,
        :payouts => Dict(
            (:C, :C) => 3.0,
            (:C, :D) => 0.0,
            (:D, :C) => 4.0,
            (:D, :D) => 1.0
            )
            )
    run_simulation(df, num_simulations, numrounds, params_for_model, "./diff_initial_distribution_of_agents/params_$(num_cooperators)_$(num_defectors)_$(num_neutral)")
    CSV.write("diff_initial_distribution_of_agents/diff_initial_distribution_of_agents.csv", df)
end

0 0 12
0 1 11
0 2 10
0 3 9
0 4 8
0 5 7
0 6 6
0 7 5
0 8 4
0 9 3
0 10 2
0 11 1
0 12 0
1 0 11
1 1 10
1 2 9
1 3 8
1 4 7
1 5 6
1 6 5
1 7 4
1 8 3
1 9 2
1 10 1
1 11 0
2 0 10
2 1 9
2 2 8
2 3 7
2 4 6
2 5 5
2 6 4
2 7 3
2 8 2
2 9 1
2 10 0
3 0 9
3 1 8
3 2 7
3 3 6
3 4 5
3 5 4
3 6 3
3 7 2
3 8 1
3 9 0
4 0 8
4 1 7
4 2 6
4 3 5
4 4 4
4 5 3
4 6 2
4 7 1
4 8 0
5 0 7
5 1 6
5 2 5
5 3 4
5 4 3
5 5 2
5 6 1
5 7 0
6 0 6
6 1 5
6 2 4
6 3 3
6 4 2
6 5 1
6 6 0
7 0 5
7 1 4
7 2 3
7 3 2
7 4 1
7 5 0
8 0 4
8 1 3
8 2 2
8 3 1
8 4 0
9 0 3
9 1 2
9 2 1
9 3 0
10 0 2
10 1 1
10 2 0
11 0 1
11 1 0
12 0 0


In [8]:
all_num_rounds = [50_000, 100_000, 500_000, 1_000_000]
num_simulations = 50
df = DataFrame(
    cai=Float64[],
    coop_degree=Float64[], 
    order_degree=Float64[], 
    numsim = Int[],
    numrounds=Int[],
    R=Float64[], 
    S=Float64[], 
    T=Float64[], 
    P=Float64[], 
    tolerance=Float64[], 
    num_defectors=Int[], 
    num_cooperators=Int[], 
    num_neutral=Int[], 
    max_positive_pressure=Float64[], 
    max_negative_pressure=Float64[], 
    adv_mutation_rate=Float64[], 
    move_mutation_rate=Float64[], 
    weight_of_present_for_adv=Float64[], 
    weight_of_present_for_move=Float64[], 
    adv_reassess_frequency=Float64[], 
    move_reassess_frequency=Float64[], 
    move_increment=Float64[]
)

for numrounds in all_num_rounds
    println("numrounds = $(numrounds)")
    params_for_model = Dict(
        :num_cooperators => 6, 
        :num_defectors => 6, 
        :num_neutral => 0,
        :max_pos_pressure => 6,
        :max_neg_pressure => 4,
        :adv_mutation_rate => 0.2,
        :move_mutation_rate => 0.15,
        :tolerance => 0.05,
        :payouts => Dict(
            (:C, :C) => 3.0,
            (:C, :D) => 0.0,
            (:D, :C) => 4.0,
            (:D, :D) => 1.0
            )
            )
    run_simulation(df, num_simulations, numrounds, params_for_model, "./diff_num_rounds/params_$(numrounds)")
    CSV.write("diff_num_rounds/diff_num_rounds.csv", df)
end

numrounds = 50000
numrounds = 100000
numrounds = 500000
numrounds = 1000000
